This notebook will create the config file and the ner model for the food
we will add this model pipeline with existing spacy pipeline in order to make a final model which should theoretically be able to identify
the food items and the amounts correctly

python -m spacy init fill-config base_config.cfg config.cfg

python -m spacy train SpaCy/TRAIN/configGPU.cfg --output SpaCy/model/  --paths.train datasets/spacyFiles/TrainData.spacy --paths.dev datasets/spacyFIles/TrainData.spacy --gpu-id 0


python -m spacy evaluate [path to model] [test.spacy path]

In [9]:
import spacy
from spacy import displacy
spacy.prefer_gpu()
# nlp = spacy.load("./SpaCy/model/model-best/")

True

In [3]:

nlp = spacy.load("en_core_web_trf",disable=["tagger","parser", "attribute_ruler", "lemmatizer"])
food_nlp = spacy.load("C:\\Users\\samad\\Documents\\DISSERTATION\\SpaCy\\model\\model-best")
food_nlp.replace_listeners("transformer", "ner", ["model.tok2vec"])

nlp.add_pipe(
    "ner",
    name="ner_food",
    source=food_nlp,
    after="ner",
)

# nlp.to_disk("../model/combined/")

In [4]:
fileloc = "../../datasets/test.txt"
with open(fileloc,'r') as file:
   TEST = file.read().split('\n')[:-1]

   TEST2 = [x.lower() for x in TEST]

In [8]:
for t in TEST[:10]:
    doc = nlp(t)
    spacy.displacy.render(doc.ents,style="ent",jupyter=True)

In [182]:
from spacy import load
import numerizer

doc = nlp("I ate three cereal bar, half of an apple and a banana for breakfast and two ham sandwiches for lunch and chicken and rice for dinner")


##testing bruteforce method
## ill get all the doc ents and if an ent is a food and the next item is also food then itll be seperate

''' 
something like  { 
    food_item : 1  (defaults as 1)
    food_item : n ( n is the value)
} 
'''     


'''  not sure its possible to change the value of an ent in the doc 
    eg. using numerizer() we convert the ent "THREE" as 3 and overwrite it in the doc.ents

    what im thinking is that we actually dont bother overwriting it and just do it real time

    maybe clean the ents list first

    so that the ents list is only cardinal and FOOD

    and say 
    if the item that we picked is a cardinal
        if the next ents is food then we use numerize on the cardinal
        and in the dict we have FOOD : cardinal._.numerize()
    if the item picked is FOOD
        and if the next ent is food then amount is 1
    and save it to dict

'''

foodDict = {  }
addedFood = []
arr = []
lastEnt = 0

i = 0 
lenEnts = len(doc.ents)
for i in range(lenEnts):
    currEnt = doc.ents[i] # the current ent
    try: 
        nextEnt = doc.ents[i+1] # the next ent in order to check
    except: 
        pass

    #if we are in a number and the next is food
    #logically thinking the number is quantity for the food 
    # while this is usual case, ofc this code can be rendered useless easily
    # because someone can say "i ate chicken wings, 20 of them"
    # and now this code will not work 😕
    
    #TODO: need to improve the ( currEnt.text not in addedFood)
    # because even still this doesnt let overwrite 
    # but if there is 2 instance of same item in the given text, 
    #   it wont be able to add it as it'll recognise that as already being in the list
    # maybe have like x,y = index,index+1 and then make sure that the index doesnt match up so no problems
    #say in index = 2 we added chicken, and index = 7 is also chicken
    #  


    #TODO: also if there were no food detected return prompt to user  
    
    if(currEnt.label_ == "CARDINAL") and nextEnt.label_ == "FOOD":
        arr.append([nextEnt.text,currEnt._.numerize()])
        addedFood.append(nextEnt.text)
        # foodDict[nextEnt.text] = (currEnt._.numerize())
    else:
        if (currEnt.label_ == "FOOD") and (currEnt.text not in addedFood) :
            # foodDict[currEnt.text] = 1 
            arr.append([currEnt.text,1])
            addedFood.append(currEnt.text)
 

print(arr)
print([n[0] for n in arr])
# print(doc.ents)



[['cereal bar', '3'], ['apple', '1/2'], ['banana', 1], ['ham sandwiches', '2'], ['chicken', 1], ['rice', 1]]
['cereal bar', 'apple', 'banana', 'ham sandwiches', 'chicken', 'rice']
